In [1]:
import pandas as pd
import ast
import re
from sqlalchemy import create_engine
from __future__ import annotations

recipes = pd.read_csv("../web_scraper/trader_joes_recipes.csv")
products = pd.read_csv("../web_scraper/trader_joes_products.csv")
fruit_veg = pd.read_csv("../web_scraper/traderjoes_fresh-fruits-veggies_products.csv")
meat = pd.read_csv("../web_scraper/traderjoes_meat_products.csv")

In [2]:
recipes["ingredients"] = recipes["ingredients"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

def split_ingredient(text):
    """
    Split strings like '4 tablespoons TJ’s Salted Butter' into:
    quantity = '4 tablespoons', ingredient = "TJ’s Salted Butter"
    """
    # Simple regex to separate numbers/units from ingredient name
    match = re.match(r"([\d¼½¾⅓⅔⅛⅜⅝⅞\s\-–/a-zA-Z]+)\s+(.*)", text)
    if match:
        qty = match.group(1).strip()
        name = match.group(2).strip()
    else:
        qty, name = None, text.strip()
    return pd.Series([qty, name])

recipes_exploded = recipes.explode("ingredients", ignore_index=True)
recipes_exploded[["quantity_text", "ingredient_name"]] = recipes_exploded["ingredients"].apply(split_ingredient)

cookbook_df = recipes_exploded[["title", "category", "ingredient_name", "quantity_text"]]
possible_ingredients = cookbook_df['ingredient_name'].unique()
possible_ingredients_df = pd.DataFrame(possible_ingredients, columns=["Ingredient"])
possible_ingredients_df

,Ingredient
0,TJ’s Salted Butter
1,TJ’s Fresh Garlic
2,TJ’s Fresh Ginger
3,"TJ’s Jasmine Rice, rinsed"
4,water
...,...
2590,"TJ's Blueberries, fresh or frozen"
2591,"TJ's Organic Basil, cut into strips"
2592,TJ's Ciliegine Mozzarella
2593,TJ's Balsamic Vinaigrette


In [3]:
# Create SQLite DB in project folder
engine = create_engine("sqlite:///cookbook.db")

# Unique ingredient and quantity lookup tables
ingredient_df = cookbook_df[["ingredient_name"]].drop_duplicates().reset_index(drop=True)
ingredient_df["ingredient_id"] = ingredient_df.index + 1

quantity_df = cookbook_df[["quantity_text"]].drop_duplicates().reset_index(drop=True)
quantity_df["quantity_id"] = quantity_df.index + 1

# Recipe table
recipe_df = recipes[["title", "category", "url", "image_url", "serves", "time"]].drop_duplicates().reset_index(drop=True)
recipe_df["recipe_id"] = recipe_df.index + 100  # just an example offset

# Link table (Cookbook)
cookbook_link = (
    cookbook_df.merge(ingredient_df, on="ingredient_name")
               .merge(quantity_df, on="quantity_text")
               .merge(recipe_df, on="title")
               [["recipe_id", "ingredient_id", "quantity_id"]]
)

# Write to SQLite
ingredient_df.to_sql("ingredient", engine, if_exists="replace", index=False)
quantity_df.to_sql("quantity", engine, if_exists="replace", index=False)
recipe_df.to_sql("recipe", engine, if_exists="replace", index=False)
cookbook_link.to_sql("recipe_ingredient", engine, if_exists="replace", index=False)

print("✅ Cookbook database built successfully.")

✅ Cookbook database built successfully.


In [5]:
# --- Option A: seed from a quick Python list (edit to your real pantry) ---
pantry_names = [
    "TJ’s Salted Butter",
    "TJ’s Fresh Garlic",
    "TJ’s Jasmine Rice",
    "TJ’s Sea Salt",
    "TJ’s Large Eggs",
    "water",
    "TJ’s Zucchini, sliced into thin planks",
    "TJ’s Crunchy Sesame Sunflower Seeds Pepitas Salsa Macha",
    "TJ’s Authentic Greek Feta In Brine, crumbled",
    "TJ’s Lemon",
    "TJ’s Cauliflower, sliced into thick planks, core intact",
    "TJ’s Hot Honey Mustard Dressing",
    "TJ’s Heirloom Tomatoes, sliced into thick rounds",
    "TJ’s Organic Ranch Dressing",
    "TJ’s Olive Oil",
    "TJ’s Angus Chuck, Brisket, & Sirloin 1/3 lb. Ground Beef Patties",
    "TJ’s Triple Cream Soft Ripened Cambozola® Blue Cheese, sliced into rectangles",
    "TJ’s Brioche Buns",
    "TJ’s Fig Butter",
    "TJ’s Arugula",
    "TJ’s Zucchini, stems removed and sliced lengthwise into thin planks"
]

# --- Option B: or load from a CSV with a single column 'ingredient_name' ---
# pantry_names = pd.read_csv("my_pantry.csv")["ingredient_name"].tolist()

# Map names -> ingredient_id using the existing ingredient table
ingredient = pd.read_sql("SELECT ingredient_id, ingredient_name FROM ingredient", engine)
pantry_df = (
    pd.DataFrame({"ingredient_name": pantry_names})
    .merge(ingredient, on="ingredient_name", how="inner")
    [["ingredient_id"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Optional: add free-text notes (e.g., amounts you actually have)
pantry_df["on_hand_note"] = None

# Write/replace the pantry table
pantry_df.to_sql("pantry", engine, if_exists="replace", index=False)

print("✅ Pantry table created with", len(pantry_df), "items")

✅ Pantry table created with 20 items


In [6]:
sql = """
SELECT
  r.recipe_id,
  r.title,
  1.0 * COUNT(p.ingredient_id) / COUNT(DISTINCT ri.ingredient_id) AS coverage
FROM recipe r
JOIN recipe_ingredient ri ON ri.recipe_id = r.recipe_id
LEFT JOIN pantry p ON p.ingredient_id = ri.ingredient_id
GROUP BY r.recipe_id, r.title
ORDER BY coverage DESC, r.title;
"""

df = pd.read_sql(sql, engine)
df_top_ten = df.head(10)
print(df_top_ten)

   recipe_id                                title  coverage
0        109  Grilled & Glazed Cauliflower Steaks  1.000000
1        108    Grilled Zucchini with Salsa Macha  1.000000
2        121            Fig & Blue Cheese Burgers  0.875000
3        110               Fried Ranch-y Tomatoes  0.500000
4        100          Aromatic Garlic Ginger Rice  0.428571
5        309          Gnocchi with Ricotta & Peas  0.375000
6        122             Zucchini & Ricotta Rolls  0.375000
7        483     Cranberry Pistachio Cheese Balls  0.333333
8        237       Maple Marshmallow Popcorn Bars  0.333333
9        576            Oat Beverage Matcha Latte  0.333333


In [7]:
sql_per_recipe = """
WITH coverage AS (
  SELECT r.recipe_id, r.title,
         1.0 * SUM(CASE WHEN p.ingredient_id IS NOT NULL THEN 1 ELSE 0 END)
           / COUNT(DISTINCT ri.ingredient_id) AS coverage
  FROM recipe r
  JOIN recipe_ingredient ri ON ri.recipe_id = r.recipe_id
  LEFT JOIN pantry p ON p.ingredient_id = ri.ingredient_id
  GROUP BY r.recipe_id, r.title
),
top10 AS (
  SELECT recipe_id, title, coverage
  FROM coverage
  ORDER BY coverage DESC, title
  LIMIT 10
),
missing AS (
  SELECT t.recipe_id, t.title, i.ingredient_name, q.quantity_text
  FROM top10 t
  JOIN recipe_ingredient ri ON ri.recipe_id = t.recipe_id
  JOIN ingredient i ON i.ingredient_id = ri.ingredient_id
  JOIN quantity   q ON q.quantity_id   = ri.quantity_id
  LEFT JOIN pantry p ON p.ingredient_id = ri.ingredient_id
  WHERE p.ingredient_id IS NULL
)
SELECT
  ingredient_name,
  REPLACE(GROUP_CONCAT(DISTINCT quantity_text), ',', ' | ') AS suggested_amounts,
  COUNT(DISTINCT recipe_id)                                   AS recipes_needing
FROM missing
GROUP BY ingredient_name
ORDER BY recipes_needing DESC, ingredient_name;
"""

sql_consolidated = """
WITH coverage AS (
  SELECT r.recipe_id, r.title,
         1.0 * SUM(CASE WHEN p.ingredient_id IS NOT NULL THEN 1 ELSE 0 END)
           / COUNT(DISTINCT ri.ingredient_id) AS coverage
  FROM recipe r
  JOIN recipe_ingredient ri ON ri.recipe_id = r.recipe_id
  LEFT JOIN pantry p ON p.ingredient_id = ri.ingredient_id
  GROUP BY r.recipe_id, r.title
),
top10 AS (
  SELECT recipe_id, title, coverage
  FROM coverage
  ORDER BY coverage DESC, title
  LIMIT 10
),
missing AS (
  SELECT t.recipe_id, t.title, i.ingredient_name, q.quantity_text
  FROM top10 t
  JOIN recipe_ingredient ri ON ri.recipe_id = t.recipe_id
  JOIN ingredient i ON i.ingredient_id = ri.ingredient_id
  JOIN quantity   q ON q.quantity_id   = ri.quantity_id
  LEFT JOIN pantry p ON p.ingredient_id = ri.ingredient_id
  WHERE p.ingredient_id IS NULL
),
-- do DISTINCT here, not inside GROUP_CONCAT
dedup_qty AS (
  SELECT ingredient_name, quantity_text
  FROM missing
  GROUP BY ingredient_name, quantity_text
),
need_counts AS (
  SELECT ingredient_name, COUNT(DISTINCT recipe_id) AS recipes_needing
  FROM missing
  GROUP BY ingredient_name
)
SELECT d.ingredient_name,
       GROUP_CONCAT(d.quantity_text, ' | ') AS suggested_amounts,
       n.recipes_needing
FROM dedup_qty d
JOIN need_counts n USING (ingredient_name)
GROUP BY d.ingredient_name
ORDER BY n.recipes_needing DESC, d.ingredient_name;
"""

missing_per_recipe = pd.read_sql(sql_per_recipe, engine)
grocery_list = pd.read_sql(sql_consolidated, engine)

print("Missing per recipe (top 10):")
# print(missing_per_recipe.head(20))
# print("\nConsolidated grocery list:")
print(grocery_list)

Missing per recipe (top 10):
                                      ingredient_name  \
0   (24.3 ounces) TJ’s Caro Sugo Italian Tomato Ba...   
1   (six ounce) bag TJ’s Mini Maple Flavored Marsh...   
2         TJ's Matcha Green Tea Single-Serve Packages   
3                         TJ's Non-Dairy Oat Beverage   
4                                TJ’s 100% Canola Oil   
5                              TJ’s Black Peppercorns   
6              TJ’s Chicken Broth Concentrate packets   
7                    TJ’s Cranberry Chevre (seasonal)   
8                                   TJ’s Cream Cheese   
9           TJ’s Dried Cranberries, very finely diced   
10  TJ’s Dry Roasted & Salted Pistachio Nutmeats, ...   
11                                  TJ’s English Peas   
12                                   TJ’s Fresh Basil   
13                      TJ’s Fresh Basil, for garnish   
14                                  TJ’s Fresh Ginger   
15                                       TJ’s Gnocchi   
16

In [8]:
# meal_plan_min_waste.py
import pandas as pd
from sqlalchemy import create_engine

DB_URL = "sqlite:///cookbook.db"   # adjust if needed

# ---------- SQL building blocks (SQLite-safe; no DISTINCT in 2-arg group_concat) ----------

SQL_MISSING_TOP10 = """
WITH coverage AS (
  SELECT r.recipe_id, r.title,
         1.0 * SUM(CASE WHEN p.ingredient_id IS NOT NULL THEN 1 ELSE 0 END)
           / COUNT(DISTINCT ri.ingredient_id) AS coverage
  FROM recipe r
  JOIN recipe_ingredient ri ON ri.recipe_id = r.recipe_id
  LEFT JOIN pantry p ON p.ingredient_id = ri.ingredient_id
  GROUP BY r.recipe_id, r.title
),
top10 AS (
  SELECT recipe_id, title, coverage
  FROM coverage
  ORDER BY coverage DESC, title
  LIMIT 10
)
SELECT
  t.recipe_id,
  t.title,
  i.ingredient_name,
  q.quantity_text
FROM top10 t
JOIN recipe_ingredient ri ON ri.recipe_id = t.recipe_id
JOIN ingredient i         ON i.ingredient_id = ri.ingredient_id
JOIN quantity   q         ON q.quantity_id   = ri.quantity_id
LEFT JOIN pantry p        ON p.ingredient_id = ri.ingredient_id
WHERE p.ingredient_id IS NULL
ORDER BY t.title, i.ingredient_name;
"""

def load_top10_missing(engine) -> pd.DataFrame:
    """
    Returns one row per missing ingredient for each of the Top-10 coverage recipes.
    Columns: recipe_id, title, ingredient_name, quantity_text
    """
    return pd.read_sql(SQL_MISSING_TOP10, engine)


# ---------- Planning utilities ----------

def rank_recipes_by_shared_missing(missing_df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds summary metrics per recipe using only the Top-10 set:
      - total_missing: count of missing ingredients for the recipe
      - shared_missing: how many of those are used by >= 2 of the Top-10 recipes
      - share_shared_missing: shared_missing / total_missing
    """
    # how many recipes (within top-10) need each ingredient
    need_counts = (
        missing_df.groupby("ingredient_name")["recipe_id"]
        .nunique()
        .rename("recipes_needing")
        .reset_index()
    )
    # join back to tag each (recipe, ingredient) as shared (>=2) or not
    tag = missing_df.merge(need_counts, on="ingredient_name", how="left")
    tag["is_shared"] = (tag["recipes_needing"] >= 2).astype(int)

    # per-recipe metrics
    per_recipe = (
        tag.groupby(["recipe_id", "title"])
           .agg(total_missing=("ingredient_name", "nunique"),
                shared_missing=("is_shared", "sum"))
           .reset_index()
    )
    per_recipe["share_shared_missing"] = (
        per_recipe["shared_missing"] / per_recipe["total_missing"].where(per_recipe["total_missing"] > 0, 1)
    )
    # high to low: prioritize recipes that share more of their missing items
    per_recipe = per_recipe.sort_values(
        by=["share_shared_missing", "shared_missing", "title"],
        ascending=[False, False, True]
    ).reset_index(drop=True)

    return per_recipe


def greedy_select_recipes(missing_df: pd.DataFrame, k: int = 4) -> pd.DataFrame:
    """
    Greedy heuristic to pick k recipes that minimize new unique ingredients introduced.
    Start from the best 'share_shared_missing' recipe, then iteratively add the recipe
    that adds the fewest new ingredients to the current selection.
    Returns a DataFrame of the selected recipes with the ranking metrics attached.
    """
    metrics = rank_recipes_by_shared_missing(missing_df)
    # map recipe -> its missing ingredient set
    recipe_sets = (
        missing_df.groupby(["recipe_id", "title"])["ingredient_name"]
        .apply(lambda s: set(s.unique()))
        .reset_index(name="missing_set")
    )

    selected_rows = []
    selected_ings: set[str] = set()

    # seed with the best-ranked recipe
    seed = metrics.iloc[0]
    selected_ids = {int(seed["recipe_id"])}
    selected_rows.append(seed)
    selected_ings |= recipe_sets.loc[
        (recipe_sets["recipe_id"] == seed["recipe_id"]), "missing_set"
    ].iloc[0]

    # iteratively pick the recipe that adds the fewest new ingredients
    while len(selected_ids) < min(k, metrics.shape[0]):
        best_row = None
        best_score = None

        for _, row in metrics.iterrows():
            rid = int(row["recipe_id"])
            if rid in selected_ids:
                continue
            ing_set = recipe_sets.loc[recipe_sets["recipe_id"] == rid, "missing_set"].iloc[0]
            new_ings = ing_set - selected_ings
            overlap = len(ing_set & selected_ings)

            # score: minimize new ingredients, break ties by larger overlap and better share_shared_missing
            score_tuple = (len(new_ings), -overlap, -row["share_shared_missing"], row["title"])

            if (best_score is None) or (score_tuple < best_score):
                best_score = score_tuple
                best_row = row

        selected_rows.append(best_row)
        selected_ids.add(int(best_row["recipe_id"]))
        selected_ings |= recipe_sets.loc[
            (recipe_sets["recipe_id"] == best_row["recipe_id"]), "missing_set"
        ].iloc[0]

    return pd.DataFrame(selected_rows).reset_index(drop=True)


def per_recipe_missing_for_selection(missing_df: pd.DataFrame, selected_ids: list[int]) -> pd.DataFrame:
    """
    Filter 'missing_df' down to the chosen recipes and return
    (recipe_id, title, ingredient_name, quantity_text), sorted nicely.
    """
    sel = (
        missing_df[missing_df["recipe_id"].isin(selected_ids)]
        .copy()
        .sort_values(by=["title", "ingredient_name"])
        .reset_index(drop=True)
    )
    return sel


def consolidated_grocery_list(missing_df: pd.DataFrame, selected_ids: list[int],
                              require_multi_use: bool = True) -> pd.DataFrame:
    """
    Build a master grocery list from the chosen recipes.
    - If require_multi_use=True, keep only ingredients used by >=2 of the chosen recipes.
    - Quantities are shown as a pipe-joined list of distinct texts (no SQLite DISTINCT bug here).
    Returns: ingredient_name | suggested_amounts | recipes_needing
    """
    sel = missing_df[missing_df["recipe_id"].isin(selected_ids)].copy()

    # count, within the selection, how many recipes need each ingredient
    need_counts_sel = (
        sel.groupby("ingredient_name")["recipe_id"]
           .nunique()
           .rename("recipes_needing")
           .reset_index()
    )
    if require_multi_use:
        keep = need_counts_sel["recipes_needing"] >= 2
        need_counts_sel = need_counts_sel[keep]

    # distinct quantities per ingredient within the selection
    qty_map = (
        sel.groupby(["ingredient_name"])["quantity_text"]
           .apply(lambda s: " | ".join(sorted(set([q for q in s.dropna().astype(str) if q.strip()]))))
           .rename("suggested_amounts")
           .reset_index()
    )

    out = need_counts_sel.merge(qty_map, on="ingredient_name", how="left")[
        ["ingredient_name", "suggested_amounts", "recipes_needing"]
    ].sort_values(by=["recipes_needing", "ingredient_name"], ascending=[False, True]).reset_index(drop=True)

    return out


# ---------- Example end-to-end run ----------

if __name__ == "__main__":
    engine = create_engine(DB_URL)

    # 1) Pull Top-10 by coverage (missing items only)
    missing_top10 = load_top10_missing(engine)
    if missing_top10.empty:
        raise SystemExit("No missing ingredients found. Did you seed your pantry and recipe tables?")

    # 2) See which recipes are best for overlap (minimize waste)
    ranked = rank_recipes_by_shared_missing(missing_top10)
    # print("\nTop-10 recipes ranked by shared-missing ratio:")
    # print(ranked[["recipe_id", "title", "shared_missing", "total_missing", "share_shared_missing"]].head(10))

    # 3) Greedy pick N recipes (change k= to your weekly plan size)
    selected = greedy_select_recipes(missing_top10, k=4)
    selected_ids = selected["recipe_id"].astype(int).tolist()
    # print("\nSelected recipes (greedy, k=4):")
    # print(selected[["recipe_id", "title", "shared_missing", "total_missing", "share_shared_missing"]])

    # 4a) Per-recipe missing checklist for the chosen set
    per_recipe = per_recipe_missing_for_selection(missing_top10, selected_ids)
    # print("\nPer-recipe missing items (chosen set):")
    # print(per_recipe.head(20))

    # 4b) Consolidated grocery list, only multi-use items (>=2 recipes)
    grocery_multi = consolidated_grocery_list(missing_top10, selected_ids, require_multi_use=True)
    print("\nConsolidated grocery list (multi-use only):")
    print(grocery_multi)

    # 4c) (Optional) If you want EVERYTHING needed (even single-use), set require_multi_use=False
    grocery_all = consolidated_grocery_list(missing_top10, selected_ids, require_multi_use=False)
    # print("\nConsolidated grocery list (all missing items):")
    # print(grocery_all)

    # (Optional) Save to CSVs for printing/sharing
    # per_recipe.to_csv("per_recipe_missing_selected.csv", index=False)
    # grocery_multi.to_csv("grocery_multi_use_selected.csv", index=False)
    # grocery_all.to_csv("grocery_all_selected.csv", index=False)


Consolidated grocery list (multi-use only):
Empty DataFrame
Columns: [ingredient_name, suggested_amounts, recipes_needing]
Index: []
